### Import libraries

In [99]:
import pandas as pd
import numpy as np

import talib as ta
import random

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
import time

### Download data

In [100]:
# import yfinance as yf

# tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA']

# start_date = "2020-01-01"
# end_date = datetime.now().strftime("%Y-%m-%d")

# # Download historical data from yf API
# data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# # download data to csv using ticker, exchange, date range and "raw" indicator in the filename
# data.to_csv(f"data_{'_'.join(tickers)}_{start_date}_{end_date}_raw.csv")

In [101]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("../00_data/data_AAPL_MSFT_GOOGL_AMZN_TSLA_META_NVDA_2020-01-01_2025-10-26_raw.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()

Ticker            MSFT                                                  \
Price             Open        High         Low       Close      Volume   
Date                                                                     
2020-01-02  151.040856  152.895807  150.612792  152.791168  22622100.0   
2020-01-03  150.603275  152.153817  150.355939  150.888641  21116200.0   
2020-01-06  149.423644  151.345190  148.881420  151.278595  20813700.0   
2020-01-07  151.554487  151.887418  149.651970  149.899292  21634100.0   
2020-01-08  151.183463  152.962326  150.251234  152.286926  27746500.0   

Ticker           AMZN                                               ...  \
Price            Open       High        Low      Close      Volume  ...   
Date                                                                ...   
2020-01-02  93.750000  94.900497  93.207497  94.900497  80580000.0  ...   
2020-01-03  93.224998  94.309998  93.224998  93.748497  75288000.0  ...   
2020-01-06  93.000000  95.184502  93.000000  95.143997  81236000.0  ...   
2020-01-07  95.224998  95.694504  94.601997  95.343002  80898000.0  ...   
2020-01-08  94.902000  95.550003  94.321999  94.598503  70160000.0  ...   

Ticker            META                                                  \
Price             Open        High         Low       Close      Volume   
Date                                                                     
2020-01-02  205.483256  208.504623  205.006201  208.494690  12077100.0   
2020-01-03  205.940433  209.110876  205.682017  207.391479  11188400.0   
2020-01-06  205.433564  211.476314  205.254674  211.297424  17058900.0   
2020-01-07  211.516088  213.265300  210.452637  211.754608  14912400.0   
2020-01-08  211.694968  214.915122  211.307358  213.901367  13475000.0   

Ticker           AAPL                                                
Price            Open       High        Low      Close       Volume  
Date                                                                 
2020-01-02  71.545890  72.598892  71.292304  72.538513  135480400.0  
2020-01-03  71.765667  72.594055  71.608685  71.833290  146322800.0  
2020-01-06  70.954203  72.444336  70.703027  72.405693  118387200.0  
2020-01-07  72.415337  72.671341  71.845369  72.065147  108872000.0  
2020-01-08  71.768101  73.526318  71.768101  73.224426  132079200.0  

[5 rows x 35 columns]

### Ticker selection

In [102]:
# random
# randomly shuffle the level 0 of the columns (the tickers), to select one random tickers for strategy and becktesting

ticker_ = random.choice(df.columns.levels[0])

print(f"Randomly selected ticker for strategy and backtesting: {ticker_}")

# # fixed
# # fixed ticker for strategy and backtesting
# ticker_ = 'AAPL'

Randomly selected ticker for strategy and backtesting: NVDA


### Data cleaning

In [103]:
# clean df to keep only the selected ticker
df = df.xs(ticker_, level=0, axis=1)

# lowercase columns
# df.columns = [col.lower() for col in df.columns]

# set index to datetime
df.index = pd.to_datetime(df.index)

# rename index to date
# df.index.name = 'date'

# show df
df.head()

Price,Open,High,Low,Close,Volume
Date,,,,,
2020-01-02,5.942537,5.971409,5.892010,5.971409,237536000.0
2020-01-03,5.851688,5.919638,5.826797,5.875831,205384000.0
2020-01-06,5.782494,5.905700,5.756359,5.900473,262636000.0
2020-01-07,5.928847,6.017705,5.883796,5.971907,314856000.0
2020-01-08,5.967677,6.024426,5.927603,5.983109,277108000.0


### Add Indicators

In [104]:
# calculate VWAP
def calculate_vwap(df):
    """Calculate VWAP for the given DataFrame."""
    vwap = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum() / df['volume'].cumsum()
    return vwap

In [105]:
# calculate ATR for stop loss and take profit
sl_multiplier = 1.5
tp_multiplier = 3.0

def calculate_atr(df, period=14):
    """Calculate ATR for the given DataFrame."""
    high_low = df['high'] - df['low']
    high_close = (df['high'] - df['close'].shift()).abs()
    low_close = (df['low'] - df['close'].shift()).abs()
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = true_range.rolling(window=period).mean()
    return atr

"""
stop_loss = entry_price - (atr * sl_multiplier)
take_profit = entry_price + (atr * tp_multiplier)
"""

'\nstop_loss = entry_price - (atr * sl_multiplier)\ntake_profit = entry_price + (atr * tp_multiplier)\n'

In [106]:
# # Define function to add indicators
# indicators = ['BBANDS', 'DEMA', 'EMA', 'KAMA', 'MA',
#               'SAR', 'MACD', 'MFI', 'MOM', 'ROC',
#               'ROCP', 'STOCH', 'RSI', 'WILLR',
#               'WMA', 'STDDEV', 'VAR', 'SMA', 'PPO'
#               ]

In [107]:
# Define Candle Pattern Conditions
candles = [
    # Bullish reversal patterns
    'CDLHAMMER',                  # Hammer
    'CDLINVERTEDHAMMER',          # Inverted Hammer
    'CDLMORNINGSTAR',             # Morning Star
    'CDLMORNINGDOJISTAR',         # Morning Doji Star
    'CDLENGULFING',               # Bullish Engulfing
    'CDLPIERCING',                # Piercing Pattern
    'CDLHARAMI',                  # Bullish Harami
    'CDLHARAMICROSS',             # Bullish Harami Cross
    'CDLTAKURI',                  # Takuri (Dragonfly Doji)
    
    # Bullish continuation patterns
    'CDL3WHITESOLDIERS',          # Three White Soldiers
    'CDLRISEFALL3METHODS',        # Rising Three Methods
    'CDLMATHOLD',                 # Mat Hold
    'CDLSEPARATINGLINES',         # Bullish Separating Lines
    'CDLTASUKIGAP',               # Bullish Tasuki Gap uptrend
    
    # Bullish bottom patterns
    'CDLABANDONEDBABY',           # Abandoned Baby
    'CDLLADDERBOTTOM',            # Ladder Bottom
    'CDLMATCHINGLOW',             # Matching Low
    'CDLUNIQUE3RIVER',            # Unique Three River
    
    # Bullish special patterns
    'CDL3INSIDE',                 # Three Inside Up
    'CDL3OUTSIDE',                # Three Outside Up
    'CDLBELTHOLD',                # Belt Hold
    'CDLBREAKAWAY',               # Breakaway
    'CDLKICKING',                 # Kicking
    'CDLKICKINGBYLENGTH',         # Kicking By Length
    'CDLSTICKSANDWICH'            # Stick Sandwich
]


In [108]:
# TWAP
# ....

### Indicators Selection

In [109]:
# Indicators (features) are splitted in two groups: indicators (list) and candles (list) 

# indicators list is already defined above
# candles list is already defined above

# First we will implement the candle patterns as features, then the indicators

In [110]:
# # interate over candles and calculate each pattern
# for candle in candles:
#     df[candle] = getattr(ta, candle)(df['open'], df['high'], df['low'], df['close'])
#     print(f"Calculated {candle}")

In [111]:
df.head()

Price,Open,High,Low,Close,Volume
Date,,,,,
2020-01-02,5.942537,5.971409,5.892010,5.971409,237536000.0
2020-01-03,5.851688,5.919638,5.826797,5.875831,205384000.0
2020-01-06,5.782494,5.905700,5.756359,5.900473,262636000.0
2020-01-07,5.928847,6.017705,5.883796,5.971907,314856000.0
2020-01-08,5.967677,6.024426,5.927603,5.983109,277108000.0


### Time Series Split

In [112]:
# add here the time series split

### Strategy and Indicators List

In [113]:
# # create class StrategyIndicators list, to hold the selected indicators and candlestick patterns for the strategy
# strategies_class_list = [
#     {
#         'name': f'VWAP_{candles[0]}',  # VWAP_CDLHAMMER
#         'primary_indicator': 'VWAP',
#         'secondary_indicator': candles[0],  # CDLHAMMER
#         'description': f'VWAP combined with {candles[0]} candlestick pattern',
#         'atr_period': 14,
#         'sl_multiplier': 1.5,
#         'tp_multiplier': 3.0
#     },
#     {
#         'name': f'VWAP_{candles[1]}',  # VWAP_CDLINVERTEDHAMMER
#         'primary_indicator': 'VWAP',
#         'secondary_indicator': candles[1],  # CDLINVERTEDHAMMER
#         'description': f'VWAP combined with {candles[1]} candlestick pattern',
#         'atr_period': 14,
#         'sl_multiplier': 1.5,
#         'tp_multiplier': 3.0
#     },
#     {
#         'name': f'VWAP_{candles[2]}',  # VWAP_CDLMORNINGSTAR
#         'primary_indicator': 'VWAP',
#         'secondary_indicator': candles[2],  # CDLMORNINGSTAR
#         'description': f'VWAP combined with {candles[2]} candlestick pattern',
#         'atr_period': 14,
#         'sl_multiplier': 1.5,
#         'tp_multiplier': 3.0
#     }
# ]

# Or create dynamically for all candle patterns:
strategies_class_list = []
for candle in candles:
    strategy_dict = {
        'name': f'VWAP_{candle}',
        'primary_indicator': 'VWAP',
        'secondary_indicator': candle,
        'description': f'VWAP combined with {candle} candlestick pattern',
        # 'atr_period': 14,
        # 'sl_multiplier': 1.5,
        # 'tp_multiplier': 3.0,
        'talib_function': getattr(ta, candle)  # Store the actual TA-Lib function
    }
    strategies_class_list.append(strategy_dict)

# Print all strategies (compact version)
print("Generated Strategy Configurations:")
for i, strategy in enumerate(strategies_class_list):
    print(f"{i+1:2d}. {strategy['name']} - {strategy['description']}")

print(f"\nTotal strategies: {len(strategies_class_list)}")

Generated Strategy Configurations:
 1. VWAP_CDLHAMMER - VWAP combined with CDLHAMMER candlestick pattern
 2. VWAP_CDLINVERTEDHAMMER - VWAP combined with CDLINVERTEDHAMMER candlestick pattern
 3. VWAP_CDLMORNINGSTAR - VWAP combined with CDLMORNINGSTAR candlestick pattern
 4. VWAP_CDLMORNINGDOJISTAR - VWAP combined with CDLMORNINGDOJISTAR candlestick pattern
 5. VWAP_CDLENGULFING - VWAP combined with CDLENGULFING candlestick pattern
 6. VWAP_CDLPIERCING - VWAP combined with CDLPIERCING candlestick pattern
 7. VWAP_CDLHARAMI - VWAP combined with CDLHARAMI candlestick pattern
 8. VWAP_CDLHARAMICROSS - VWAP combined with CDLHARAMICROSS candlestick pattern
 9. VWAP_CDLTAKURI - VWAP combined with CDLTAKURI candlestick pattern
10. VWAP_CDL3WHITESOLDIERS - VWAP combined with CDL3WHITESOLDIERS candlestick pattern
11. VWAP_CDLRISEFALL3METHODS - VWAP combined with CDLRISEFALL3METHODS candlestick pattern
12. VWAP_CDLMATHOLD - VWAP combined with CDLMATHOLD candlestick pattern
13. VWAP_CDLSEPARATINGL

In [114]:
strategies_class_list[:5]

[{'name': 'VWAP_CDLHAMMER',
  'primary_indicator': 'VWAP',
  'secondary_indicator': 'CDLHAMMER',
  'description': 'VWAP combined with CDLHAMMER candlestick pattern',
  'talib_function': <function talib._ta_lib.CDLHAMMER(open, high, low, close)>},
 {'name': 'VWAP_CDLINVERTEDHAMMER',
  'primary_indicator': 'VWAP',
  'secondary_indicator': 'CDLINVERTEDHAMMER',
  'description': 'VWAP combined with CDLINVERTEDHAMMER candlestick pattern',
  'talib_function': <function talib._ta_lib.CDLINVERTEDHAMMER(open, high, low, close)>},
 {'name': 'VWAP_CDLMORNINGSTAR',
  'primary_indicator': 'VWAP',
  'secondary_indicator': 'CDLMORNINGSTAR',
  'description': 'VWAP combined with CDLMORNINGSTAR candlestick pattern',
  'talib_function': <function talib._ta_lib.CDLMORNINGSTAR(open, high, low, close, penetration=0.3)>},
 {'name': 'VWAP_CDLMORNINGDOJISTAR',
  'primary_indicator': 'VWAP',
  'secondary_indicator': 'CDLMORNINGDOJISTAR',
  'description': 'VWAP combined with CDLMORNINGDOJISTAR candlestick pattern

### Backtest

In [115]:
# Add necessary imports for backtesting
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

# Strategy implementation using your existing functions and data
class VWAPEngulfingStrategy(Strategy):
    """
    VWAP + Bullish Engulfing Strategy with ATR-based Stop Loss and Take Profit
    
    Entry Conditions:
    (no crossover, price simply above VWAP)!
    - Price is above VWAP (bullish signal)
    - Bullish Engulfing pattern detected
    
    Exit Conditions:
    - Stop Loss: entry_price - (atr * sl_multiplier)
    - Take Profit: entry_price + (atr * tp_multiplier)
    """
    
    # Strategy parameters (using your existing values)
    atr_period = 14
    sl_multiplier = 1.5
    tp_multiplier = 3.0
    
    def init(self):
        """Initialize indicators using your existing functions"""
        # Calculate VWAP using your calculate_vwap function logic
        typical_price = (self.data.High + self.data.Low + self.data.Close) / 3
        volume_price = typical_price * self.data.Volume
        self.vwap = self.I(lambda: volume_price.cumsum() / self.data.Volume.cumsum(), name='VWAP')
        
        # Calculate ATR using talib (as in your notebook)
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_period, name='ATR')
        
        # Calculate Bullish Engulfing pattern using talib
        self.engulfing = self.I(ta.CDLENGULFING, self.data.Open, self.data.High, 
                               self.data.Low, self.data.Close, name='Engulfing')
        
        # Track entry levels
        self.entry_price = None
        self.stop_loss_level = None
        self.take_profit_level = None
        
    def next(self):
        """Define trading logic"""
        # Entry conditions
        if (not self.position and 
            self.data.Close[-1] > self.vwap[-1] and  # Price above VWAP
            self.engulfing[-1] > 0):  # Bullish engulfing pattern (100)
            
            # Enter long position
            self.buy()
            
            # Set entry price and levels using your multipliers
            self.entry_price = self.data.Close[-1]
            self.stop_loss_level = self.entry_price - (self.atr[-1] * self.sl_multiplier)
            self.take_profit_level = self.entry_price + (self.atr[-1] * self.tp_multiplier)
        
        # Exit conditions
        if self.position:
            # Stop Loss
            if self.data.Low[-1] <= self.stop_loss_level:
                self.position.close()
                self.entry_price = None
                
            # Take Profit
            elif self.data.High[-1] >= self.take_profit_level:
                self.position.close()
                self.entry_price = None

# Prepare data for backtesting (ensure proper column names)
# The backtesting library expects uppercase column names
df_backtest = df.copy()
df_backtest.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

# Run backtest on your actual data
bt = Backtest(df_backtest, VWAPEngulfingStrategy, 
              cash=10000, commission=0.002)

# Execute backtest
results = bt.run()

# Display results
print("="*60)
print("VWAP + BULLISH ENGULFING STRATEGY BACKTEST RESULTS")
print(f"Ticker: {ticker_}")
print(f"Period: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")
print("="*60)

# Key performance metrics
key_metrics = [
    'Return [%]',
    'Buy & Hold Return [%]',
    'Max Drawdown [%]',
    'Volatility (ann.) [%]',
    'Sharpe Ratio',
    '# Trades',
    'Win Rate [%]',
    'Best Trade [%]',
    'Worst Trade [%]',
    'Avg Trade [%]'
]

for metric in key_metrics:
    if metric in results.index:
        value = results[metric]
        if isinstance(value, (int, float)):
            if '%' in metric:
                print(f"{metric:<25}: {value:>8.2f}%")
            elif metric == '# Trades':
                print(f"{metric:<25}: {value:>8.0f}")
            else:
                print(f"{metric:<25}: {value:>8.4f}")

print(f"\nStrategy Parameters:")
print(f"{'ATR Period':<25}: {VWAPEngulfingStrategy.atr_period}")
print(f"{'Stop Loss Multiplier':<25}: {VWAPEngulfingStrategy.sl_multiplier}")
print(f"{'Take Profit Multiplier':<25}: {VWAPEngulfingStrategy.tp_multiplier}")

VWAP + BULLISH ENGULFING STRATEGY BACKTEST RESULTS
Ticker: NVDA
Period: 2020-01-02 to 2025-10-24
Return [%]               :    78.77%
Buy & Hold Return [%]    :  2859.45%
Sharpe Ratio             :   0.4608
# Trades                 :       27
Win Rate [%]             :    51.85%
Best Trade [%]           :    18.35%
Worst Trade [%]          :    -9.20%

Strategy Parameters:
ATR Period               : 14
Stop Loss Multiplier     : 1.5
Take Profit Multiplier   : 3.0


In [116]:
# Additional comprehensive backtest metrics
print("\n" + "="*80)
print("COMPREHENSIVE PERFORMANCE METRICS")
print("="*80)

# Performance ratios and risk metrics
print(f"\nRISK-ADJUSTED PERFORMANCE:")
print(f"{'Calmar Ratio':<30}: {results.get('Calmar Ratio', 'N/A'):>12}")
print(f"{'Sortino Ratio':<30}: {results.get('Sortino Ratio', 'N/A'):>12}")
print(f"{'Alpha [%]':<30}: {results.get('Alpha [%]', 'N/A'):>12.2f}")
print(f"{'Beta':<30}: {results.get('Beta', 'N/A'):>12.4f}")

# Drawdown analysis
print(f"\nDRAWDOWN ANALYSIS:")
print(f"{'Avg. Drawdown [%]':<30}: {results.get('Avg. Drawdown [%]', 'N/A'):>12.2f}")

# Handle Timedelta objects for duration metrics
max_dd_duration = results.get('Max. Drawdown Duration', 'N/A')
avg_dd_duration = results.get('Avg. Drawdown Duration', 'N/A')

# Convert Timedelta to string if it's not 'N/A'
if max_dd_duration != 'N/A' and hasattr(max_dd_duration, 'days'):
    max_dd_str = f"{max_dd_duration.days} days" if max_dd_duration.days > 0 else str(max_dd_duration)
else:
    max_dd_str = str(max_dd_duration)

if avg_dd_duration != 'N/A' and hasattr(avg_dd_duration, 'days'):
    avg_dd_str = f"{avg_dd_duration.days} days" if avg_dd_duration.days > 0 else str(avg_dd_duration)
else:
    avg_dd_str = str(avg_dd_duration)

print(f"{'Max. Drawdown Duration':<30}: {max_dd_str:>12}")
print(f"{'Avg. Drawdown Duration':<30}: {avg_dd_str:>12}")

# Trade analysis
print(f"\nTRADE ANALYSIS:")
print(f"{'Profit Factor':<30}: {results.get('Profit Factor', 'N/A'):>12.2f}")
print(f"{'Expectancy [%]':<30}: {results.get('Expectancy [%]', 'N/A'):>12.2f}")
print(f"{'SQN (System Quality Number)':<30}: {results.get('SQN', 'N/A'):>12.2f}")
print(f"{'Kelly Criterion':<30}: {results.get('Kelly Criterion', 'N/A'):>12.4f}")

# Duration metrics - Handle Timedelta objects
print(f"\nTRADE DURATION:")

max_trade_duration = results.get('Max. Trade Duration', 'N/A')
avg_trade_duration = results.get('Avg. Trade Duration', 'N/A')

# Convert Timedelta to string if it's not 'N/A'
if max_trade_duration != 'N/A' and hasattr(max_trade_duration, 'days'):
    max_trade_str = f"{max_trade_duration.days} days" if max_trade_duration.days > 0 else str(max_trade_duration)
else:
    max_trade_str = str(max_trade_duration)

if avg_trade_duration != 'N/A' and hasattr(avg_trade_duration, 'days'):
    avg_trade_str = f"{avg_trade_duration.days} days" if avg_trade_duration.days > 0 else str(avg_trade_duration)
else:
    avg_trade_str = str(avg_trade_duration)

print(f"{'Max. Trade Duration':<30}: {max_trade_str:>12}")
print(f"{'Avg. Trade Duration':<30}: {avg_trade_str:>12}")
print(f"{'Exposure Time [%]':<30}: {results.get('Exposure Time [%]', 'N/A'):>12.2f}")

# Additional performance metrics
print(f"\nADDITIONAL METRICS:")
print(f"{'Return (Ann.) [%]':<30}: {results.get('Return (Ann.) [%]', 'N/A'):>12.2f}")
print(f"{'CAGR [%]':<30}: {results.get('CAGR [%]', 'N/A'):>12.2f}")

# Handle equity peak formatting
equity_peak = results.get('Equity Peak [$]', 'N/A')
if equity_peak != 'N/A':
    print(f"{'Equity Peak [$]':<30}: {equity_peak:>12,.2f}")
else:
    print(f"{'Equity Peak [$]':<30}: {'N/A':>12}")

# Strategy classification based on SQN
sqn = results.get('SQN', 0)
if sqn != 'N/A' and not pd.isna(sqn):
    if sqn >= 2.5:
        sqn_rating = "Excellent"
    elif sqn >= 1.9:
        sqn_rating = "Good"
    elif sqn >= 1.4:
        sqn_rating = "Average"
    elif sqn >= 1.0:
        sqn_rating = "Below Average"
    else:
        sqn_rating = "Poor"
    
    print(f"\nSTRATEGY QUALITY RATING:")
    print(f"{'SQN Rating':<30}: {sqn_rating:>12} (SQN: {sqn:.2f})")
else:
    print(f"\nSTRATEGY QUALITY RATING:")
    print(f"{'SQN Rating':<30}: {'N/A':>12} (SQN: N/A)")

# Performance summary
total_return = results.get('Return [%]', 0)
win_rate = results.get('Win Rate [%]', 0)
max_dd = results.get('Max Drawdown [%]', 0)
sharpe = results.get('Sharpe Ratio', 0)

print(f"\nPERFORMANCE SUMMARY:")
if total_return != 'N/A' and not pd.isna(total_return):
    print(f"Strategy shows {'POSITIVE' if total_return > 0 else 'NEGATIVE'} returns with {win_rate:.1f}% win rate")
    print(f"Risk level: {'HIGH' if abs(max_dd) > 20 else 'MODERATE' if abs(max_dd) > 10 else 'LOW'} (Max DD: {abs(max_dd):.1f}%)")
    if sharpe != 'N/A' and not pd.isna(sharpe):
        print(f"Risk-adjusted performance: {'EXCELLENT' if sharpe > 2 else 'GOOD' if sharpe > 1 else 'POOR'} (Sharpe: {sharpe:.2f})")
    else:
        print(f"Risk-adjusted performance: N/A (Sharpe: N/A)")
else:
    print("No trades executed - insufficient data for performance analysis")

print("="*80)


COMPREHENSIVE PERFORMANCE METRICS

RISK-ADJUSTED PERFORMANCE:
Calmar Ratio                  : 0.3104205130000225
Sortino Ratio                 : 0.7863021450974279
Alpha [%]                     :      -328.63
Beta                          :       0.1425

DRAWDOWN ANALYSIS:
Avg. Drawdown [%]             :        -5.93
Max. Drawdown Duration        :    1318 days
Avg. Drawdown Duration        :      99 days

TRADE ANALYSIS:
Profit Factor                 :         1.91
Expectancy [%]                :         2.47
SQN (System Quality Number)   :         1.55
Kelly Criterion               :       0.2502

TRADE DURATION:
Max. Trade Duration           :      42 days
Avg. Trade Duration           :      16 days
Exposure Time [%]             :        21.00

ADDITIONAL METRICS:
Return (Ann.) [%]             :        10.53
CAGR [%]                      :         7.14
Equity Peak [$]               :    17,876.99

STRATEGY QUALITY RATING:
SQN Rating                    :      Average (SQN: 1.55)

P

In [117]:
# backtest out of sample data

In [118]:
# create performance report with the backtest results for all strategies after out of sample backtest

In [119]:
# optimize the parameters of the 3 best strategies
"""
def optimize
(
self,
*,
maximize='SQN',
method='grid',
max_tries=None,
constraint=None,
return_heatmap=False,
return_optimization=False,
random_state=None,
**kwargs)
Optimize strategy parameters to an optimal combination. Returns result pd.Series of the best run.

maximize is a string key from the Backtest.run()-returned results series, or a function that accepts this series object and returns a number; the higher the better. By default, the method maximizes Van Tharp's System Quality Number.

method is the optimization method. Currently two methods are supported:

"grid" which does an exhaustive (or randomized) search over the cartesian product of parameter combinations, and
"sambo" which finds close-to-optimal strategy parameters using model-based optimization, making at most max_tries evaluations.
max_tries is the maximal number of strategy runs to perform. If method="grid", this results in randomized grid search. If max_tries is a floating value between (0, 1], this sets the number of runs to approximately that fraction of full grid space. Alternatively, if integer, it denotes the absolute maximum number of evaluations. If unspecified (default), grid search is exhaustive, whereas for method="sambo", max_tries is set to 200.

constraint is a function that accepts a dict-like object of parameters (with values) and returns True when the combination is admissible to test with. By default, any parameters combination is considered admissible.

If return_heatmap is True, besides returning the result series, an additional pd.Series is returned with a multiindex of all admissible parameter combinations, which can be further inspected or projected onto 2D to plot a heatmap (see plot_heatmaps()).

If return_optimization is True and method = 'sambo', in addition to result series (and maybe heatmap), return raw scipy.optimize.OptimizeResult for further inspection, e.g. with SAMBO's plotting tools.

If you want reproducible optimization results, set random_state to a fixed integer random seed.

Additional keyword arguments represent strategy arguments with list-like collections of possible values. For example, the following code finds and returns the "best" of the 7 admissible (of the 9 possible) parameter combinations:

best_stats = backtest.optimize(sma1=[5, 10, 15], sma2=[10, 20, 40],
                               constraint=lambda p: p.sma1 < p.sma2)
"""

'\ndef optimize\n(\nself,\n*,\nmaximize=\'SQN\',\nmethod=\'grid\',\nmax_tries=None,\nconstraint=None,\nreturn_heatmap=False,\nreturn_optimization=False,\nrandom_state=None,\n**kwargs)\nOptimize strategy parameters to an optimal combination. Returns result pd.Series of the best run.\n\nmaximize is a string key from the Backtest.run()-returned results series, or a function that accepts this series object and returns a number; the higher the better. By default, the method maximizes Van Tharp\'s System Quality Number.\n\nmethod is the optimization method. Currently two methods are supported:\n\n"grid" which does an exhaustive (or randomized) search over the cartesian product of parameter combinations, and\n"sambo" which finds close-to-optimal strategy parameters using model-based optimization, making at most max_tries evaluations.\nmax_tries is the maximal number of strategy runs to perform. If method="grid", this results in randomized grid search. If max_tries is a floating value between 

In [120]:
# 3 best strategies after optimization should be tested on 1 hour data

In [ ]:
from matplotlib.dates import DateFormatter
import seaborn as sns

def plot_backtest_results(results, df_backtest, ticker_name, strategy_name, 
                         plot_equity=True, plot_drawdown=True, 
                         plot_trades=True, plot_performance=True,
                         figsize=(15, 12)):
    """
    Comprehensive plotting function for backtesting results visualization
    
    Parameters:
    -----------
    results : pd.Series
        Backtest results from backtesting library
    df_backtest : pd.DataFrame
        OHLCV data used for backtesting
    ticker_name : str
        Name of the ticker being analyzed
    strategy_name : str
        Name of the strategy
    plot_equity : bool
        Whether to plot equity curve
    plot_drawdown : bool
        Whether to plot drawdown
    plot_trades : bool
        Whether to plot trade markers
    plot_performance : bool
        Whether to plot performance metrics
    figsize : tuple
        Figure size (width, height)
    """
    
    # Set style
    plt.style.use('seaborn-v0_8')
    
    # Create subplots
    if plot_performance:
        fig = plt.figure(figsize=figsize)
        gs = fig.add_gridspec(3, 2, height_ratios=[2, 1, 1], hspace=0.3, wspace=0.3)
        
        # Main price and equity plot
        ax1 = fig.add_subplot(gs[0, :])
        # Drawdown plot
        ax2 = fig.add_subplot(gs[1, :], sharex=ax1)
        # Performance metrics
        ax3 = fig.add_subplot(gs[2, 0])
        ax4 = fig.add_subplot(gs[2, 1])
    else:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, 
                                      height_ratios=[3, 1], sharex=True)
    
    # Colors
    colors = {
        'price': '#1f77b4',
        'equity': '#ff7f0e', 
        'drawdown': '#d62728',
        'buy': '#2ca02c',
        'sell': '#ff1744',
        'background': '#f8f9fa'
    }
    
    # Plot 1: Price and Equity
    ax1_twin = ax1.twinx()
    
    # Price data
    ax1.plot(df_backtest.index, df_backtest['Close'], 
             color=colors['price'], linewidth=1.5, label=f'{ticker_name} Price', alpha=0.8)
    
    if plot_equity and hasattr(results, '_trades') and len(results._trades) > 0:
        # Calculate equity curve from trades
        trades = results._trades
        equity_curve = calculate_equity_curve(trades, initial_cash=10000)
        
        ax1_twin.plot(equity_curve.index, equity_curve.values, 
                     color=colors['equity'], linewidth=2, label='Portfolio Equity')
    
    # Plot trade markers if available
    if plot_trades and hasattr(results, '_trades') and len(results._trades) > 0:
        trades = results._trades
        
        # Entry points
        entry_points = trades[trades['Size'] > 0]
        if not entry_points.empty:
            ax1.scatter(entry_points['EntryTime'], 
                       entry_points['EntryPrice'],
                       color=colors['buy'], marker='^', 
                       s=100, label='Buy Signal', zorder=5)
        
        # Exit points
        exit_points = trades[trades['Size'] > 0]
        if not exit_points.empty:
            ax1.scatter(exit_points['ExitTime'], 
                       exit_points['ExitPrice'],
                       color=colors['sell'], marker='v', 
                       s=100, label='Sell Signal', zorder=5)
    
    ax1.set_title(f'{strategy_name} - {ticker_name} Backtest Results', 
                  fontsize=16, fontweight='bold', pad=20)
    ax1.set_ylabel('Price ($)', fontsize=12)
    ax1_twin.set_ylabel('Portfolio Value ($)', fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='upper left')
    ax1_twin.legend(loc='upper right')
    
    # Plot 2: Drawdown
    if plot_drawdown:
        drawdown_data = calculate_drawdown(results)
        ax2.fill_between(drawdown_data.index, drawdown_data.values, 0,
                        color=colors['drawdown'], alpha=0.6, label='Drawdown')
        ax2.plot(drawdown_data.index, drawdown_data.values, 
                color=colors['drawdown'], linewidth=1)
        
        ax2.set_ylabel('Drawdown (%)', fontsize=12)
        ax2.set_xlabel('Date', fontsize=12)
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        # Format x-axis
        ax2.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
    
    # Plot 3 & 4: Performance Metrics
    if plot_performance:
        # Key metrics table
        metrics_data = {
            'Metric': ['Total Return', 'Sharpe Ratio', 'Max Drawdown', 
                      'Win Rate', '# Trades', 'Volatility'],
            'Value': [
                f"{results.get('Return [%]', 0):.2f}%",
                f"{results.get('Sharpe Ratio', 0):.2f}",
                f"{results.get('Max Drawdown [%]', 0):.2f}%",
                f"{results.get('Win Rate [%]', 0):.2f}%",
                f"{int(results.get('# Trades', 0))}",
                f"{results.get('Volatility (ann.) [%]', 0):.2f}%"
            ]
        }
        
        ax3.axis('tight')
        ax3.axis('off')
        table = ax3.table(cellText=[[m, v] for m, v in zip(metrics_data['Metric'], metrics_data['Value'])],
                         colLabels=['Metric', 'Value'],
                         cellLoc='center',
                         loc='center',
                         colWidths=[0.6, 0.4])
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax3.set_title('Key Performance Metrics', fontweight='bold', pad=20)
        
        # Returns distribution
        if hasattr(results, '_trades') and len(results._trades) > 0:
            trades = results._trades
            returns = ((trades['ExitPrice'] - trades['EntryPrice']) / trades['EntryPrice'] * 100).dropna()
            
            if len(returns) > 0:
                ax4.hist(returns, bins=min(20, len(returns)//2 + 1), 
                        alpha=0.7, color=colors['equity'], edgecolor='black')
                ax4.axvline(returns.mean(), color='red', linestyle='--', 
                           label=f'Mean: {returns.mean():.2f}%')
                ax4.set_xlabel('Trade Return (%)')
                ax4.set_ylabel('Frequency')
                ax4.set_title('Trade Returns Distribution', fontweight='bold')
                ax4.grid(True, alpha=0.3)
                ax4.legend()
    
    plt.tight_layout()
    return fig

def calculate_equity_curve(trades, initial_cash=10000):
    """Calculate equity curve from trades dataframe"""
    if trades.empty:
        return pd.Series([initial_cash], index=[pd.Timestamp.now()])
    
    equity = initial_cash
    equity_curve = [equity]
    dates = [trades.iloc[0]['EntryTime']]
    
    for _, trade in trades.iterrows():
        if trade['Size'] > 0:  # Long position
            pnl = (trade['ExitPrice'] - trade['EntryPrice']) * trade['Size']
            equity += pnl
            equity_curve.append(equity)
            dates.append(trade['ExitTime'])
    
    return pd.Series(equity_curve, index=dates)

def calculate_drawdown(results):
    """Calculate drawdown series from results"""
    # This is a simplified version - in practice, you'd get this from the backtest
    # For demonstration, creating mock drawdown data
    if hasattr(results, '_trades') and len(results._trades) > 0:
        equity = calculate_equity_curve(results._trades)
        running_max = equity.expanding().max()
        drawdown = ((equity - running_max) / running_max * 100)
        return drawdown
    else:
        # Return empty series if no trades
        return pd.Series([], dtype=float)

# Enhanced plotting function with additional features
def plot_strategy_comparison(results_list, strategy_names, ticker_name, figsize=(15, 8)):
    """
    Compare multiple strategies performance
    
    Parameters:
    -----------
    results_list : list of pd.Series
        List of backtest results
    strategy_names : list of str
        Names of strategies
    ticker_name : str
        Name of the ticker
    """
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=figsize)
    
    # Metrics comparison
    metrics = ['Return [%]', 'Sharpe Ratio', 'Max Drawdown [%]', 'Win Rate [%]']
    comparison_data = {}
    
    for metric in metrics:
        comparison_data[metric] = [results.get(metric, 0) for results in results_list]
    
    # Bar plots for comparison
    x_pos = np.arange(len(strategy_names))
    
    # Returns comparison
    ax1.bar(x_pos, comparison_data['Return [%]'], alpha=0.7, color='skyblue')
    ax1.set_title('Total Returns Comparison')
    ax1.set_ylabel('Return (%)')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(strategy_names, rotation=45)
    ax1.grid(True, alpha=0.3)
    
    # Sharpe Ratio comparison
    ax2.bar(x_pos, comparison_data['Sharpe Ratio'], alpha=0.7, color='lightgreen')
    ax2.set_title('Sharpe Ratio Comparison')
    ax2.set_ylabel('Sharpe Ratio')
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(strategy_names, rotation=45)
    ax2.grid(True, alpha=0.3)
    
    # Max Drawdown comparison (negative values)
    ax3.bar(x_pos, [-abs(dd) for dd in comparison_data['Max Drawdown [%]']], 
            alpha=0.7, color='salmon')
    ax3.set_title('Max Drawdown Comparison')
    ax3.set_ylabel('Max Drawdown (%)')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(strategy_names, rotation=45)
    ax3.grid(True, alpha=0.3)
    
    # Win Rate comparison
    ax4.bar(x_pos, comparison_data['Win Rate [%]'], alpha=0.7, color='gold')
    ax4.set_title('Win Rate Comparison')
    ax4.set_ylabel('Win Rate (%)')
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels(strategy_names, rotation=45)
    ax4.grid(True, alpha=0.3)
    
    plt.suptitle(f'Strategy Performance Comparison - {ticker_name}', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    return fig

In [122]:
# Monte Carlo simulation or bootstrap to evaluate the robustness of the strategies
"""
def random_ohlc_data
(
example_data, *, frac=1.0, random_state=None)
OHLC data generator. The generated OHLC data has basic descriptive statistics similar to the provided example_data.

frac is a fraction of data to sample (with replacement). Values greater than 1 result in oversampling.

Such random data can be effectively used for stress testing trading strategy robustness, Monte Carlo simulations, significance testing, etc.

>>> from backtesting.test import EURUSD
>>> ohlc_generator = random_ohlc_data(EURUSD)
>>> next(ohlc_generator)  # returns new random data
...
>>> next(ohlc_generator)  # returns new random data
...
def resample_apply
(
rule, func, series, *args, agg=None, **kwargs)
Apply func (such as an indicator) to series, resampled to a time frame specified by rule. When called from inside Strategy.init(), the result (returned) series will be automatically wrapped in Strategy.I() wrapper method.

rule is a valid Pandas offset string indicating a time frame to resample series to.

func is the indicator function to apply on the resampled series.

series is a data series (or array), such as any of the Strategy.data series. Due to pandas resampling limitations, this only works when input series has a datetime index.

agg is the aggregation function to use on resampled groups of data. Valid values are anything accepted by pandas/resample/.agg(). Default value for dataframe input is OHLCV_AGG dictionary. Default value for series input is the appropriate entry from OHLCV_AGG if series has a matching name, or otherwise the value "last", which is suitable for closing prices, but you might prefer another (e.g. "max" for peaks, or similar).

Finally, any *args and **kwargs that are not already eaten by implicit Strategy.I() call are passed to func.

For example, if we have a typical moving average function SMA(values, lookback_period), hourly data source, and need to apply the moving average MA(10) on a daily time frame, but don't want to plot the resulting indicator, we can do:

class System(Strategy):
    def init(self):
        self.sma = resample_apply(
            'D', SMA, self.data.Close, 10, plot=False)
The above short snippet is roughly equivalent to:

class System(Strategy):
    def init(self):
        # Strategy exposes <code>self.data</code> as raw NumPy arrays.
        # Let's convert closing prices back to pandas Series.
        close = self.data.Close.s

        # Resample to daily resolution. Aggregate groups
        # using their last value (i.e. closing price at the end
        # of the day). Notice `label='right'`. If it were set to
        # 'left' (default), the strategy would exhibit
        # look-ahead bias.
        daily = close.resample('D', label='right').agg('last')

        # We apply SMA(10) to daily close prices,
        # then reindex it back to original hourly index,
        # forward-filling the missing values in each day.
        # We make a separate function that returns the final
        # indicator array.
        def SMA(series, n):
            from backtesting.test import SMA
            return SMA(series, n).reindex(close.index).ffill()

        # The result equivalent to the short example above:
        self.sma = self.I(SMA, daily, 10, plot=False)
    """

'\ndef random_ohlc_data\n(\nexample_data, *, frac=1.0, random_state=None)\nOHLC data generator. The generated OHLC data has basic descriptive statistics similar to the provided example_data.\n\nfrac is a fraction of data to sample (with replacement). Values greater than 1 result in oversampling.\n\nSuch random data can be effectively used for stress testing trading strategy robustness, Monte Carlo simulations, significance testing, etc.\n\n>>> from backtesting.test import EURUSD\n>>> ohlc_generator = random_ohlc_data(EURUSD)\n>>> next(ohlc_generator)  # returns new random data\n...\n>>> next(ohlc_generator)  # returns new random data\n...\ndef resample_apply\n(\nrule, func, series, *args, agg=None, **kwargs)\nApply func (such as an indicator) to series, resampled to a time frame specified by rule. When called from inside Strategy.init(), the result (returned) series will be automatically wrapped in Strategy.I() wrapper method.\n\nrule is a valid Pandas offset string indicating a time f

In [123]:
"""
class MultiBacktest
(
df_list, strategy_cls, **kwargs)
Multi-dataset Backtest wrapper.

Run supplied Strategy on several instruments, in parallel. Used for comparing strategy runs across many instruments or classes of instruments. Example:

from backtesting.test import EURUSD, BTCUSD, SmaCross
btm = MultiBacktest([EURUSD, BTCUSD], SmaCross)
stats_per_ticker: pd.DataFrame = btm.run(fast=10, slow=20)
heatmap_per_ticker: pd.DataFrame = btm.optimize(...)
Methods
def optimize
(
self, **kwargs)
Wraps Backtest.optimize(), but returns pd.DataFrame with currency indexes in columns.

heamap: pd.DataFrame = btm.optimize(...)
from backtesting.plot import plot_heatmaps
plot_heatmaps(heatmap.mean(axis=1))
def run
(
self, **kwargs)
Wraps Backtest.run(). Returns pd.DataFrame with currency indexes in columns.
"""

'\nclass MultiBacktest\n(\ndf_list, strategy_cls, **kwargs)\nMulti-dataset Backtest wrapper.\n\nRun supplied Strategy on several instruments, in parallel. Used for comparing strategy runs across many instruments or classes of instruments. Example:\n\nfrom backtesting.test import EURUSD, BTCUSD, SmaCross\nbtm = MultiBacktest([EURUSD, BTCUSD], SmaCross)\nstats_per_ticker: pd.DataFrame = btm.run(fast=10, slow=20)\nheatmap_per_ticker: pd.DataFrame = btm.optimize(...)\nMethods\ndef optimize\n(\nself, **kwargs)\nWraps Backtest.optimize(), but returns pd.DataFrame with currency indexes in columns.\n\nheamap: pd.DataFrame = btm.optimize(...)\nfrom backtesting.plot import plot_heatmaps\nplot_heatmaps(heatmap.mean(axis=1))\ndef run\n(\nself, **kwargs)\nWraps Backtest.run(). Returns pd.DataFrame with currency indexes in columns.\n'

In [124]:
# Final performance report